### 前言
@Z.Liang, 20190809
<br>最近在读《用python进行数据分析第二版》。第89页看到defaultdict时，觉得有点意思。
<br>于是，结合自己以前写的脚本，对比总结。

### dict有很多方法，其中get(), setdefault()比较有意思。
<br>

|方法|含义|
|-----|-----|
|.get(key,arg)|```lambda x: dict[key] if dict[key] != None else arg```|
|.setdefault(key,arg)|```dict[key] if dict[key] != None else dict[key]=arg```|

In [1]:
words = ['apple', 'bear', 'arguement', 'banana','cocona', 'cyan', 'apple', 'bear', 'apple', 'bear', 'cocona', 'cyan', 'apple', 'bear', 'cocona', 'cyan']

In [2]:
d = {}
for word in words:
    letter = word[0]
    d.setdefault(letter, []).append(word)

In [3]:
d

{'a': ['apple', 'arguement', 'apple', 'apple', 'apple'],
 'b': ['bear', 'banana', 'bear', 'bear', 'bear'],
 'c': ['cocona', 'cyan', 'cocona', 'cyan', 'cocona', 'cyan']}

In [4]:
for k, v in d.items():
    print(k, v)

a ['apple', 'arguement', 'apple', 'apple', 'apple']
b ['bear', 'banana', 'bear', 'bear', 'bear']
c ['cocona', 'cyan', 'cocona', 'cyan', 'cocona', 'cyan']


## 计数方法一：常规方法
>1. 字典化(关键字:计数) d={}
>2. 将字典的元素list化 li = list(d.items())
>4. 排序li.sort(lambda x: x[1], reverse=True) 
>5. 然后读取top n

```for i in range(10):
       k, v = li[i]
       print("{0}{1}".format(k, v))
```

In [5]:
counter_d = {}
for word in words:
    counter_d[word] = counter_d.get(word, 0) + 1

In [6]:
counter_d

{'apple': 4, 'bear': 4, 'arguement': 1, 'banana': 1, 'cocona': 3, 'cyan': 3}

In [7]:
li = list(counter_d.items())
li.sort(key=lambda x: x[1], reverse=True)
li

[('apple', 4),
 ('bear', 4),
 ('cocona', 3),
 ('cyan', 3),
 ('arguement', 1),
 ('banana', 1)]

In [8]:
for i in range(5):
    k, v = li[i]
    print("{0:<10}{1:>5}".format(k, v))

apple         4
bear          4
cocona        3
cyan          3
arguement     1


## 延伸：collections
<br>collections模块里有一个数据类型叫<font color=green>defaultdict<font color=black>, 接受一个类型并初始化字典 [hight-performance container datatypes](https://docs.python.org/2/library/collections.html)

|数据类型|含义|版本|实例|
|---|---|---|---|
|namedtuple()|factory function for creating **tuple** subclasses with named fields|New in version 2.6|带有名字的元组，该元组的元素通过名字(而不是索引)进行获取|
|deque|**list-like** container with fast appends and pops on either end|New in version 2.4.||
|Counter|**dict** subclass for counting hashable objects|New in version 2.7.|用于计数的字典|
|OrderedDict|**dict** subclass that remembers the order entries were added|New in version 2.7.|用于记忆元素顺序的字典|
|defaultdict|**dict** subclass that calls a factory function to supply missing values|New in version 2.5.|带有初始化功能的字典|

In [9]:
from collections import Counter, defaultdict

## 计数方法二：collections.Counter([n])
用于计数，美滋滋
    
>1. Counter()对象，并计数
>2. Counter().most_common([n]), 返回top n的元素 

In [10]:
cnt = Counter()
for word in words:
    cnt[word] += 1

In [11]:
for k, v in cnt.most_common():
    print(k, v)

apple 4
bear 4
cocona 3
cyan 3
arguement 1
banana 1


## 计数方法三: 特殊字典collections.defaultdict(seq)

In [12]:
dd = defaultdict(int)

In [13]:
for word in words:
    dd[word] += 1

In [14]:
dd

defaultdict(int,
            {'apple': 4,
             'bear': 4,
             'arguement': 1,
             'banana': 1,
             'cocona': 3,
             'cyan': 3})

In [43]:
new_li = list(new_d.items())
new_li

[('a', ['apple', 'arguement']),
 ('b', ['bear', 'banana']),
 ('c', ['cocona', 'cyan'])]

## 小结：计数方法对比

|数据结构|方法|使用技巧|
|--|--|--|
|常规字典|d.get(key, arg)|创建空字典->填充元素(关键字:计数)->将元素列表化，并排序->用for循环输出top n|
|特殊字典|collections.defaultdict(seq)|创建字典->填充元素并计数<font color=blue>此步简化<font color=black>->将元素列表化，并排序->用for循环输出top n|
|特殊字典(计数器)|collections.Counter()|创建计数器->填充元素(关键字:计数)->Counter().most_common(n)输出top n|

**速度和效果**：特殊字典(计数器) > 特殊字典defaultdict > 常规字典

## 应用: 词云
手头有一部《<font color=red>红楼梦<font color=black>》的电子版，想要统计主要人物名字在书中出现的频次，并显示为词云
<br>思路和实现：
- step1.读取源数据。可能涉及到的问题：汉字解码decoding
- step2.词汇/名字截断。可能涉及到的问题：如何实现？ [jieba](https://pypi.org/project/jieba/)
- step3.主要人物名字计数。可能涉及到的问题：根据汉字文化，截断后的词汇长度为1，或者长度大于5，均可不计入
- step4.非主要人物名字排除, 并生成结果。
- step5.显示为词云。[wordcloud](https://pypi.org/project/wordcloud/)

In [15]:
from wordcloud import WordCloud as wc
import jieba as jb

In [16]:
hlm = r"C:\Users\5106001995\Desktop\2019_pypj\_17.word_cloud\hlm.txt"
with open(hlm, encoding='utf-8', errors='ignore') as f:
    s = f.read()

In [17]:
w = jb.lcut(s)
d = Counter()

for i in w:
    if len(i) != 1:
        d[i] += 1

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\510600~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.761 seconds.
Prefix dict has been built succesfully.


In [18]:
exc = (['什么', '一个', '我们', '那里', '你们', '如今', '知道', '说道', '起来', '姑娘', '这里',
        '出来', '他们', '众人', '自己', '一面', '太太', '只见', '怎么', '奶奶', '两个', '没有',
        '不是', '不知', '这个', '听见', '这样', '进来', '咱们', '告诉', '就是', '东西', '回来',
        '只是', '只得', '老爷', '大家', '丫头', '这些', '不敢', '出去', '所以', '不过', '的话',
        '不好', '姐姐', '一时', '不能', '过来', '心里', '二爷', '如此', '今日', '银子', '几个',
        '答应', '二人', '还有', '只管', '老太太', '这么', '说话', '一回', '那边', '这话', '外头',
        '打发', '自然', '今儿', '罢了', '屋里', '那些', '听说', '小丫头'
        ])

for i in exc:
    del(d[i])

In [19]:
for k, v in d.most_common(15):
    print("{0:<10} {1:>5}".format(k, v))

宝玉          3748
贾母          1227
凤姐          1099
王夫人         1011
贾琏           670
平儿           588
袭人           585
宝钗           567
黛玉           560
凤姐儿          470
薛姨妈          453
探春           432
鸳鸯           425
贾政           350
晴雯           336


In [20]:
nw = ''
for k, v in d.most_common(15):
    nw = nw + k + ' '
wdcd = (wc(
            background_color='black',
            width=400,
            height=320,
            font_path='msyh.ttc',
            max_words=15,
            max_font_size=80,
            stopwords=exc
        ).generate(nw))

In [22]:
import os 
to_filepath = os.path.join(os.path.split(hlm)[0], 'hlmcy.png')
wdcd.to_file(to_filepath)
os.startfile(to_filepath)